In [7]:
from __future__ import absolute_import, division, print_function
import os, sys, json
current_path = os.path.abspath('.')
parent_path = os.path.dirname(current_path)
sys.path.append(parent_path)
import argparse
import collections,math
import time,datetime,pytz
import numpy as np
import pandas as pd
import pickle
import unittest

from scipy.fftpack import dct, idct
from scipy import stats
from scipy import interpolate, io
from scipy.io import loadmat
from collections import Counter
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
import operator
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torchvision import datasets, transforms
from tqdm import tqdm
from torchinfo import summary
import faiss

from src.data_loader import *
from src.utils import *

from src.models.simple_cnn import *
from src.models.resnet_models import *
from src.models.dscnn import *
from train_joint import *

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

############################################
########## Plot Style Declaration ##########
# Set the style globally
# Alternatives include bmh, fivethirtyeight, ggplot,
# dark_background, seaborn-deep, etc
# plt.style.use('ggplot')
plt.style.use('seaborn-white')

plt.rcParams['font.family'] = 'times new roman'
# plt.rcParams['font.serif'] = 'Ubuntu'
# plt.rcParams['font.monospace'] = 'Ubuntu Mono'
plt.rcParams['font.size'] = 15
plt.rcParams['axes.labelsize'] = 15
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['axes.titlesize'] = 15
plt.rcParams['xtick.labelsize'] = 15
plt.rcParams['ytick.labelsize'] = 15
plt.rcParams['legend.fontsize'] = 14
plt.rcParams['figure.titlesize'] = 15
plt.rcParams['lines.linewidth'] = 3
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

##################################
########## End of Setup ##########

In [15]:
class ArgClass():
    def __init__(self):
        super(ArgClass, self).__init__()

args = ArgClass()
args.dataset="mnist"
args.model_arch = "cnn"
args.batch_size=128
args.test_batch_size=1000
args.percent=[0.8, 0.92, 0.991, 0.93]
args.alpha=5e-4
args.rho=1e-2
args.l1=False
args.l2=False
args.num_pre_epochs=3
args.num_epochs=5
args.num_re_epochs=3
args.lr=1e-3
args.adam_epsilon=1e-8
args.no_cuda=True
args.seed=1
args.save_model=False
args.shuffle=True

args.optimizer_name = 'adam'
args.lr_mode = 'multistep'
args.lr_decay = 0.1
args.lr_decay_epoch = '20,40'
args.target_lr = 1e-8
args.warmup_epochs = 0
args.warmup_lr = 1e-8
args.warmup_mode = 'linear'
args.momentum = 0.9
args.wd = 0.0001
args.gamma_wd_mult = 1.0
args.beta_wd_mult = 1.0
args.bias_wd_mult = 1.0
args.grad_clip = None
args.label_smoothing = False


args.test_fold_l = '[10]'
args.use_one_task = 'false'
args.exp_setup = ''
args.subject_idx = None
args.pretrained=False
args.best_acc = 200.0
args.mixup = False
args.mixup_alpha = 1.0
args.mixup_epoch_tail = 10
args.session = 1
args.test_vote = None

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [16]:
use_cuda

False

In [17]:
from torch.quantization import QuantStub, DeQuantStub
class PTCVDSCNNQuant(nn.Module):
    def __init__(self, model_name="dscnn_l", in_channels=1, num_classes=12, init_block_kernel=(10,4), pretrained=False, **kwargs):
        super(PTCVDSCNNQuant, self).__init__()
        self.model = get_dscnn(model_name=model_name.lower(),
                               in_channels=in_channels,
                               num_classes=num_classes,
                               init_block_kernel=init_block_kernel,
                               pretrained=pretrained,
                               **kwargs)
        self.quant = QuantStub()
        self.dequant = DeQuantStub()
        
    def forward(self, x):
        x = self.quant(x)
        x = self.model(x)
        x = self.dequant(x)
        return F.log_softmax(x, dim=1)


class AudioCNNQuant(nn.Module):
    def __init__(self,
                 channels=[32,32,64,64],
                 in_channels=1,
                 in_size=(32, 32),
                 num_classes=10,
                 pooling_type='avg'
                 ):
        super(AudioCNNQuant, self).__init__()

        self.conv1 = nn.Sequential(
                        nn.Conv2d(in_channels,channels[0],kernel_size=(3, 3),stride=1,padding=1),
                        nn.BatchNorm2d(channels[0]),
                        nn.ReLU(inplace=True),
                    )
        self.conv2 = nn.Sequential(
                        nn.Conv2d(channels[0], channels[1], kernel_size=(3, 3), stride=1, padding=1),
                        nn.BatchNorm2d(channels[1]),
                        nn.ReLU(inplace=True),
                        nn.MaxPool2d(2, stride=2) if pooling_type == 'max' else nn.AvgPool2d(2, stride=2)
                    )
        self.conv3 = nn.Sequential(
                        nn.Conv2d(channels[1], channels[2], kernel_size=(3, 3), stride=1, padding=1),
                        nn.BatchNorm2d(channels[2]),
                        nn.ReLU(inplace=True),
                    )
        self.conv4 = nn.Sequential(
                        nn.Conv2d(channels[2], channels[3], kernel_size=(3, 3), stride=1, padding=1),
                        nn.BatchNorm2d(channels[3]),
                        nn.ReLU(inplace=True),
                        nn.MaxPool2d(2, stride=2) if pooling_type == 'max' else nn.AvgPool2d(2, stride=2)
                    )

        self.dropout = nn.Dropout(0.5)
        features_dim, seq_dim = get_feature_seq_dim(4, in_size)
        self.fc = nn.Linear(channels[3]*features_dim*seq_dim, num_classes)
        self.quant = QuantStub()
        self.dequant = DeQuantStub()

    def forward(self, x):
        x = self.quant(x)
        
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.dropout(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.dropout(x)
        # x = x.view(x.size(0), -1)
        x = x.reshape(x.size(0), -1)
        x = self.fc(x)
        
        x = self.dequant(x)
        return F.log_softmax(x, dim=1)

    
class VWNetQuant(nn.Module):
    def __init__(self,
                 args,
                 in_channels=1,
                 in_size=(32, 32),
                 channels=[32, 32, 64, 64],
                 fcDims=[128, 128],
                 kernels=[(3,3)],
                 num_classes=10,
                 pooling_type='avg'
                 ):
        super(VWNetQuant, self).__init__()

        self.features = nn.Sequential()
        self.features.add_module("init_block",
             nn.Sequential(
                 nn.Conv2d(in_channels, channels[0], kernel_size=kernels[0], stride=1,padding=0),
                 nn.ReLU(inplace=True),
                 nn.MaxPool2d(2, stride=2) if pooling_type == 'max' else nn.AvgPool2d(2, stride=2))
         )

        for i, channels_per_stage in enumerate(channels):
            if i == 0:
                continue
            self.features.add_module("stage{}".format(i),
                 nn.Sequential(
                     nn.Conv2d(channels[i-1], channels[i], kernel_size=kernels[i] if len(kernels)>1 else kernels[0], stride=1,padding=0),
                     nn.ReLU(inplace=True),
                     nn.MaxPool2d(2, stride=2) if pooling_type == 'max' else nn.AvgPool2d(2, stride=2))
             )

        self.dropout = nn.Dropout(0.5)
        features_dim, seq_dim = get_feature_seq_dim_nopadding(len(channels), in_size, kernels)

        self.decoder = nn.Sequential()
        self.decoder.add_module("init_decoder",
            nn.Sequential(
                nn.Linear(channels[-1] * features_dim * seq_dim, fcDims[0]),
                nn.ReLU(inplace=True),
                nn.Dropout(0.5))
        )

        if len(fcDims) > 1:
            for i, channels_per_stage in enumerate(fcDims):
                if i == 0:
                    continue
                self.decoder.add_module("decode_stage{}".format(i),
                    nn.Sequential(
                        nn.Linear(fcDims[i-1], fcDims[i]),
                        nn.ReLU(inplace=True),
                        nn.Dropout(0.5))
                )

        self.output = nn.Linear(fcDims[-1], num_classes)
        self.quant = QuantStub()
        self.dequant = DeQuantStub()


    def forward(self, x):
        x = self.quant(x)
        x = self.features(x)
        # print(x.size())
        x = self.dropout(x)
        x = x.view(x.size(0), -1)
        x = self.decoder(x)
        x = self.output(x)
        x = self.dequant(x)
        return F.log_softmax(x, dim=1)

    


In [22]:
# 90.05
args.config = DATASET_CONFIGS['cifar10']
cifar10 = PTCVDSCNNQuant(model_name='MicroNet-AD-I-ch168', in_channels=args.config["in_channels"],
                             num_classes=args.config["classes"],
                             init_block_kernel=(3, 3),
                             pretrained=False)
cifar10.load_state_dict(torch.load('../data/saved_model/MicroNet-AD-I-ch168_cifar10_adam_multistep_nomixup.pt', map_location=device))

# 94.48
args.config = DATASET_CONFIGS['svhn']
svhn = PTCVDSCNNQuant(model_name='MicroNet-AD-I-ch168', in_channels=args.config["in_channels"],
                             num_classes=args.config["classes"],
                             init_block_kernel=(3, 3),
                             pretrained=False)
svhn.load_state_dict(torch.load('../data/saved_model/MicroNet-AD-I-ch168_svhn_adam_multistep_nomixup.pt', map_location=device))

# 97.50
args.config = DATASET_CONFIGS['gtsrb-vw']
gtsrb = PTCVDSCNNQuant(model_name='MicroNet-AD-I-ch168', in_channels=args.config["in_channels"],
                             num_classes=args.config["classes"],
                             init_block_kernel=(3, 3),
                             pretrained=False)
gtsrb.load_state_dict(torch.load('../data/saved_model/MicroNet-AD-I-ch168_gtsrb-vw_adam_multistep_nomixup.pt', map_location=device))

# 78.9
config = DATASET_CONFIGS['hhar-noaug']
hhar = PTCVDSCNNQuant(model_name='MicroNet-AD-I-ch176-s41', in_channels=1,
                             num_classes=config["classes"],
                             init_block_kernel=(3, 3),
                             pretrained=args.pretrained)
hhar.load_state_dict(torch.load('../data/saved_model/MicroNet-hhar-ch176_hhar-noaug_adam_multistep_nomixup.pt', map_location=device))



# 73.0
config = DATASET_CONFIGS['stl10']
stl10 = PTCVDSCNNQuant(model_name='DS-CNN-M-s33', in_channels=config["in_channels"],
                             num_classes=config["classes"],
                             init_block_kernel=(3, 3),
                             pretrained=args.pretrained)
stl10.load_state_dict(torch.load('../data/saved_model/DS-CNN-M-s33_stl10_adam_multistep_nomixup.pt', map_location=device))

# 93.47
config = DATASET_CONFIGS['urbansound8k-LMCST-long-randfold']
urbansound = PTCVDSCNNQuant(model_name='DS-CNN-M-s33', in_channels=1,
                             num_classes=config["classes"],
                             init_block_kernel=(3, 3),
                             pretrained=args.pretrained)
urbansound.load_state_dict(torch.load('../data/saved_model/DS-CNN-M-s33_urbansound8k-LMCST-long-randfold_adam_multistep_nomixup.pt', map_location=device))


# Unseen HAR
# 91.5
config = DATASET_CONFIGS['pamap2']
pamap2 = PTCVDSCNNQuant(model_name='MicroNet-AD-I-ch168-s21', in_channels=1,
                             num_classes=config["classes"],
                             init_block_kernel=(3, 3),
                             pretrained=args.pretrained)
pamap2.load_state_dict(torch.load('../data/saved_model/MicroNet-AD-I-ch168-s21_pamap2_adam_multistep_nomixup.pt', map_location=device))

# 97.65
config = DATASET_CONFIGS['skoda']
skoda = PTCVDSCNNQuant(model_name='MicroNet-AD-I-ch192-s22', in_channels=1,
                             num_classes=config["classes"],
                             init_block_kernel=(3, 3),
                             pretrained=args.pretrained)
skoda.load_state_dict(torch.load('../data/saved_model/MicroNet-AD-I-ch192-s22_skoda_adam_multistep_nomixup.pt', map_location=device))



<All keys matched successfully>

In [11]:
# Lee's datasets
# args.dataset = 'mnist'
# train_loader_mnist, test_loader_mnist = get_data_loaders(args, kwargs)
args.dataset='cifar10'
train_loader_cifar10, test_loader_cifar10 = get_data_loaders(args, kwargs)
args.dataset='svhn'
train_loader_svhn, test_loader_svhn = get_data_loaders(args, kwargs)
args.dataset='gtsrb-vw'
train_loader_gtsrb, test_loader_gtsrb = get_data_loaders(args, kwargs)
# args.dataset='gscv2-mfcc10'
# train_loader_gsc, test_loader_gsc = get_data_loaders(args, kwargs)

# additional datasets to be trained 
args.dataset = 'hhar-noaug'
train_loader_hhar, test_loader_hhar = get_data_loaders(args, kwargs)
# args.dataset = 'ninapro-db2-c10'
# args.subject_idx=11
# args.exp_setup='per-subject'
# train_loader_ninadb2, test_loader_ninadb2 = get_data_loaders(args, kwargs)

# unseen datasets
# args.dataset = 'fashion-mnist'
# train_loader_fmnist, test_loader_fmnist = get_data_loaders(args, kwargs)
args.dataset = 'stl10'
train_loader_stl10, test_loader_stl10 = get_data_loaders(args, kwargs)

# args.dataset = 'emotion-5-frames25'
# train_loader_emotion, test_loader_emotion = get_data_loaders(args, kwargs)
args.dataset = 'urbansound8k-LMCST-long-randfold'
train_loader_urbansound, test_loader_urbansound = get_data_loaders(args, kwargs)

args.dataset = 'pamap2'
train_loader_pamap2, test_loader_pamap2 = get_data_loaders(args, kwargs)
args.dataset = 'skoda'
train_loader_skoda, test_loader_skoda = get_data_loaders(args, kwargs)

# args.dataset = 'ninapro-db3-c10'
# args.subject_idx=1
# train_loader_ninadb3, test_loader_ninadb3 = get_data_loaders(args, kwargs)
# args.dataset = 'ninapro-db6-c7-seq80'
# train_loader_ninadb6, test_loader_ninadb6 = get_data_loaders(args, kwargs)

Files already downloaded and verified
Files already downloaded and verified
Using downloaded and verified file: ../data-link/image/svhn/train_32x32.mat
Using downloaded and verified file: ../data-link/image/svhn/test_32x32.mat
X.shape:  (39209, 1, 32, 32)
y.shape:  (39209,)
X.shape:  (12630, 1, 32, 32)
y.shape:  (12630,)
X.shape:  (59403, 1, 20, 120)
y.shape:  (59403,)
X.shape:  (7721, 1, 20, 120)
y.shape:  (7721,)
Files already downloaded and verified
Files already downloaded and verified
X.shape:  (7859, 1, 128, 85)
y.shape:  (7859,)
X.shape:  (873, 1, 128, 85)
y.shape:  (873,)
The shape of the train dataset is 52, and the labels is 473445
The shape of the val dataset is 52, and the labels is 90814
The shape of the test dataset is 52, and the labels is 83366
X.shape:  (35263, 1, 33, 52)
y.shape:  (35263,)
X.shape:  (5209, 1, 33, 52)
y.shape:  (5209,)
The shape of the train dataset is 60, and the labels is 143825
The shape of the val dataset is 60, and the labels is 16976
The shape of

## helper functions

In [19]:
import torch.quantization
def print_size_of_model(model):
    torch.save(model.state_dict(), "temp.p")
    model_size = os.path.getsize("temp.p")/1e6
    print('Size (MB):', model_size)
    os.remove('temp.p')
    return model_size

def get_size_of_model(model):
    torch.save(model.state_dict(), "temp.p")
    model_size = os.path.getsize("temp.p")/1e6
    os.remove('temp.p')
    return model_size

def train(args, model, device, train_loader, test_loader, optimizer):
    args.best_acc = -0.1
    for epoch in range(args.num_epochs):
        print(f'Epoch: {epoch+1:5d}')
        args.epoch = epoch + 1
        model.train()
        for batch_idx, (data, target) in enumerate(tqdm(train_loader)):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            if args.mixup and (epoch < args.num_epochs - args.mixup_epoch_tail) :
                data, target_a, target_b, lam = mixup_data(data, target, device, alpha=args.mixup_alpha)
                output = model(data)
                loss = mixup_loss(args, model, output, target_a, target_b, lam)
            else:
                output = model(data)
                loss = regularized_nll_loss(args, model, output, target)
            loss.backward()
            optimizer.step()
        train_acc = test(args, model, device, train_loader, 3)
        test_acc = test(args, model, device, test_loader, 3)
        if train_acc > 70.:
            break

def test(args, model, device, test_loader, neval_batches):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for cnt, (data, target) in enumerate(test_loader):
            if args.test_vote is None:
                data, target = data.to(device), target.to(device)
                output = model(data)
                test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
                pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
                correct += pred.eq(target.view_as(pred)).sum().item()
            elif args.test_vote == 'avg_pool_vote':
                data, target = data.to(device), target.to(device)
                tot_window_size = data.size()[3] # NCHW
                window_size = args.config['seq']
                window_stride = args.config['seq']
                for i in range(0, tot_window_size, window_stride):
                    data_temp = data[:,:,:,i*window_stride:i*window_stride+window_size] if i*window_stride+window_size <= tot_window_size else data[:,:,:,-1-window_size:-1]
                    output_temp = model(data_temp)
                    test_loss += F.nll_loss(output_temp, target, reduction='sum').item()  # sum up batch loss
                    if i == 0:
                        output = output_temp
                    else:
                        output += output_temp
                pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
                correct += pred.eq(target.view_as(pred)).sum().item()
            if cnt >= neval_batches:
                neval = (cnt+1)*data.size(0)
                test_loss /= neval
                
                print('\nTrain or Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
                test_loss, correct, neval,
                100. * correct / neval))
                return 100. * correct / neval

    test_loss /= len(test_loader.dataset)

    print('\nTrain or Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    if args.best_acc < 100. * correct / len(test_loader.dataset):
        args.best_acc = 100. * correct / len(test_loader.dataset)
        args.best_acc_epoch = args.epoch
        print(f'New Best Accuracy: ({args.best_acc:.2f}%) at Epoch: ({args.best_acc_epoch})\n')
    return 100. * correct / len(test_loader.dataset)
        
def staticQuantizePerTensor(args, rModel, train_loader, test_loader, num_calibration_batches=32,num_eval_batches=1000):
    
    myModel = copy.deepcopy(rModel.to('cpu'))
    myModel.eval()
    size_prev = get_size_of_model(myModel)

    # Specify quantization configuration
    # Start with simple min/max range estimation and per-tensor quantization of weights
    myModel.qconfig = torch.quantization.default_qconfig
    print(myModel.qconfig)
    torch.quantization.prepare(myModel, inplace=True)

    # Calibrate with the training set
    acc_prev = test(args, myModel.to(device), device, train_loader, num_calibration_batches)
    print('Post Training Quantization: Calibration done')

    # Convert to quantized model
    torch.quantization.convert(myModel, inplace=True)
    print('Post Training Quantization: Convert done')

    size_curr = get_size_of_model(myModel)

    acc_curr = test(args, myModel.to(device), device, test_loader, num_eval_batches)
    print(f'Size Prev (MB): {size_prev}, Size Current (MB): {size_curr}, Comp Rate: {size_prev/size_curr} ')
    return acc_prev, acc_curr, size_prev/size_curr

def getResult4Quant(args, rModel, train_loader, test_loader, num_calibration_batches=32,num_eval_batches=1000):
    ret_l = []
#     acc_curr_l = []
#     comp_rate_l = []
    for i in range(5):
        torch.manual_seed(i)
        ret = staticQuantizePerTensor(args,rModel,train_loader,test_loader,num_calibration_batches,num_eval_batches)
        ret_l.append(ret)
    ret_l = np.array(ret_l)
    print(ret_l)
    print(f'Accuracy: {np.mean(ret_l[:,1])}, Acc_Std: {np.std(ret_l[:,1])}, Comp Rate Avg.: {np.mean(ret_l[:,2])}, Comp Rate Std.: {np.std(ret_l[:,2])}')

    
def staticQuantizePerChannel(args, rModel, train_loader, test_loader, num_calibration_batches=32,num_eval_batches=1000):
    
    myModel = copy.deepcopy(rModel.to('cpu'))
    myModel.eval()
    size_prev = get_size_of_model(myModel)

    # Specify quantization configuration
    # Start with simple min/max range estimation and per-tensor quantization of weights
    myModel.qconfig = torch.quantization.get_default_qconfig('fbgemm')
    print(myModel.qconfig)
    torch.quantization.prepare(myModel, inplace=True)

    # Calibrate with the training set
    acc_prev = test(args, myModel.to(device), device, train_loader, num_calibration_batches)
    print('Post Training Quantization: Calibration done')

    # Convert to quantized model
    torch.quantization.convert(myModel, inplace=True)
    print('Post Training Quantization: Convert done')

    size_curr = get_size_of_model(myModel)

    acc_curr = test(args, myModel.to(device), device, test_loader, num_eval_batches)
    print(f'Size Prev (MB): {size_prev}, Size Current (MB): {size_curr}, Comp Rate: {size_prev/size_curr} ')
    return acc_prev, acc_curr, size_prev/size_curr

def getResult4QuantChannel(args, rModel, train_loader, test_loader, num_calibration_batches=32,num_eval_batches=1000):
    ret_l = []
#     acc_curr_l = []
#     comp_rate_l = []
    for i in range(5):
        torch.manual_seed(i)
        ret = staticQuantizePerChannel(args,rModel,train_loader,test_loader,num_calibration_batches,num_eval_batches)
        ret_l.append(ret)
    ret_l = np.array(ret_l)
    print(ret_l)
    print(f'Accuracy: {np.mean(ret_l[:,1])}, Acc_Std: {np.std(ret_l[:,1])}, Comp Rate Avg.: {np.mean(ret_l[:,2])}, Comp Rate Std.: {np.std(ret_l[:,2])}')
    

def qaQuantizePerChannel(args, rModel, train_loader, test_loader, num_calibration_batches=32,num_eval_batches=1000):
    
    myModel = copy.deepcopy(rModel.to(device))
    myModel.eval()
    size_prev = get_size_of_model(myModel)

    # Specify quantization configuration
    # Start with simple min/max range estimation and per-tensor quantization of weights
    # optimizer = torch.optim.SGD(myModel.parameters(), lr = 0.001)
    optimizer = torch.optim.Adam(myModel.parameters(), lr = 0.001, betas=(0.9, 0.999),weight_decay=0.0001)
    
    myModel.qconfig = torch.quantization.get_default_qat_qconfig('fbgemm')
    print(myModel.qconfig)
    torch.quantization.prepare_qat(myModel, inplace=True)

    # Calibrate with the training set
    acc_prev = test(args, myModel.to(device), device, train_loader, num_calibration_batches)
    print('Post Training Quantization: Calibration done')
    
    for nepoch in range(3):
        train(args, myModel.to(device), device, train_loader, test_loader, optimizer)
        
        if nepoch > 3:
            # Freeze quantizer parameters
            myModel.apply(torch.quantization.disable_observer)
        if nepoch > 2:
            # Freeze batch norm mean and variance estimates
            myModel.apply(torch.nn.intrinsic.qat.freeze_bn_stats)
        
        quantized_model = torch.quantization.convert(myModel.to('cpu').eval(), inplace=False)
        quantized_model.eval()
        acc_curr = test(args, quantized_model.to('cpu'), 'cpu', test_loader, 10)
        print(f'MY Current Accuracy: {acc_curr}, at Epoch: {nepoch}')
    
    # Convert to quantized model
    torch.quantization.convert(myModel.to('cpu'), inplace=True)
    print('Post Training Quantization: Convert done')

    size_curr = get_size_of_model(myModel)

    acc_curr = test(args, myModel.to('cpu'), 'cpu', test_loader, num_eval_batches)
    print(f'Size Prev (MB): {size_prev}, Size Current (MB): {size_curr}, Comp Rate: {size_prev/size_curr} ')
    return acc_prev, acc_curr, size_prev/size_curr

def getResult4qaQuantChannel(args, rModel, train_loader, test_loader, num_calibration_batches=32,num_eval_batches=1000):
    ret_l = []
#     acc_curr_l = []
#     comp_rate_l = []
    for i in range(5):
        torch.manual_seed(i)
        ret = qaQuantizePerChannel(args,rModel,train_loader,test_loader,num_calibration_batches,num_eval_batches)
        ret_l.append(ret)
    ret_l = np.array(ret_l)
    print(ret_l)
    print(f'Accuracy: {np.mean(ret_l[:,1])}, Acc_Std: {np.std(ret_l[:,1])}, Comp Rate Avg.: {np.mean(ret_l[:,2])}, Comp Rate Std.: {np.std(ret_l[:,2])}')
    

## Results below (Seen Models)
num_calibration_batches = 32
num_eval_batches = 10000

In [23]:
getResult4QuantChannel(args,cifar10,train_loader_cifar10,test_loader_cifar10,32,10000)

QConfig(activation=functools.partial(<class 'torch.quantization.observer.HistogramObserver'>, reduce_range=True), weight=functools.partial(<class 'torch.quantization.observer.PerChannelMinMaxObserver'>, dtype=torch.qint8, qscheme=torch.per_channel_symmetric))

Train or Test set: Average loss: 0.0758, Accuracy: 4123/4224 (97.61%)
Post Training Quantization: Calibration done
Post Training Quantization: Convert done

Train or Test set: Average loss: 0.3926, Accuracy: 8901/10000 (89.01%)
Size Prev (MB): 0.837056, Size Current (MB): 0.302188, Comp Rate: 2.769984248216342 
QConfig(activation=functools.partial(<class 'torch.quantization.observer.HistogramObserver'>, reduce_range=True), weight=functools.partial(<class 'torch.quantization.observer.PerChannelMinMaxObserver'>, dtype=torch.qint8, qscheme=torch.per_channel_symmetric))

Train or Test set: Average loss: 0.0814, Accuracy: 4117/4224 (97.47%)
Post Training Quantization: Calibration done
Post Training Quantization: Convert done

Train or

In [24]:
getResult4QuantChannel(args,svhn,train_loader_svhn,test_loader_svhn,32,10000)


QConfig(activation=functools.partial(<class 'torch.quantization.observer.HistogramObserver'>, reduce_range=True), weight=functools.partial(<class 'torch.quantization.observer.PerChannelMinMaxObserver'>, dtype=torch.qint8, qscheme=torch.per_channel_symmetric))

Train or Test set: Average loss: 0.1233, Accuracy: 4075/4224 (96.47%)
Post Training Quantization: Calibration done
Post Training Quantization: Convert done

Train or Test set: Average loss: 0.2200, Accuracy: 24422/26032 (93.82%)
Size Prev (MB): 0.837056, Size Current (MB): 0.30216, Comp Rate: 2.7702409319565797 
QConfig(activation=functools.partial(<class 'torch.quantization.observer.HistogramObserver'>, reduce_range=True), weight=functools.partial(<class 'torch.quantization.observer.PerChannelMinMaxObserver'>, dtype=torch.qint8, qscheme=torch.per_channel_symmetric))

Train or Test set: Average loss: 0.1332, Accuracy: 4064/4224 (96.21%)
Post Training Quantization: Calibration done
Post Training Quantization: Convert done

Train o

In [25]:
getResult4QuantChannel(args,gtsrb,train_loader_gtsrb,test_loader_gtsrb,32,10000)


QConfig(activation=functools.partial(<class 'torch.quantization.observer.HistogramObserver'>, reduce_range=True), weight=functools.partial(<class 'torch.quantization.observer.PerChannelMinMaxObserver'>, dtype=torch.qint8, qscheme=torch.per_channel_symmetric))

Train or Test set: Average loss: 0.0004, Accuracy: 4224/4224 (100.00%)
Post Training Quantization: Calibration done
Post Training Quantization: Convert done

Train or Test set: Average loss: 0.0940, Accuracy: 12306/12630 (97.43%)
Size Prev (MB): 0.850406, Size Current (MB): 0.306054, Comp Rate: 2.7786142314754914 
QConfig(activation=functools.partial(<class 'torch.quantization.observer.HistogramObserver'>, reduce_range=True), weight=functools.partial(<class 'torch.quantization.observer.PerChannelMinMaxObserver'>, dtype=torch.qint8, qscheme=torch.per_channel_symmetric))

Train or Test set: Average loss: 0.0004, Accuracy: 4224/4224 (100.00%)
Post Training Quantization: Calibration done
Post Training Quantization: Convert done

Trai

In [26]:
getResult4QuantChannel(args,hhar,train_loader_hhar,test_loader_hhar,32,10000)

QConfig(activation=functools.partial(<class 'torch.quantization.observer.HistogramObserver'>, reduce_range=True), weight=functools.partial(<class 'torch.quantization.observer.PerChannelMinMaxObserver'>, dtype=torch.qint8, qscheme=torch.per_channel_symmetric))

Train or Test set: Average loss: 0.0005, Accuracy: 4224/4224 (100.00%)
Post Training Quantization: Calibration done
Post Training Quantization: Convert done

Train or Test set: Average loss: 1.5056, Accuracy: 5409/7721 (70.06%)
Size Prev (MB): 0.829154, Size Current (MB): 0.300604, Comp Rate: 2.758293302816995 
QConfig(activation=functools.partial(<class 'torch.quantization.observer.HistogramObserver'>, reduce_range=True), weight=functools.partial(<class 'torch.quantization.observer.PerChannelMinMaxObserver'>, dtype=torch.qint8, qscheme=torch.per_channel_symmetric))

Train or Test set: Average loss: 0.0006, Accuracy: 4224/4224 (100.00%)
Post Training Quantization: Calibration done
Post Training Quantization: Convert done

Train o

In [27]:
getResult4QuantChannel(args,stl10,train_loader_stl10,test_loader_stl10,32,10000)

QConfig(activation=functools.partial(<class 'torch.quantization.observer.HistogramObserver'>, reduce_range=True), weight=functools.partial(<class 'torch.quantization.observer.PerChannelMinMaxObserver'>, dtype=torch.qint8, qscheme=torch.per_channel_symmetric))

Train or Test set: Average loss: 0.3035, Accuracy: 3815/4224 (90.32%)
Post Training Quantization: Calibration done
Post Training Quantization: Convert done

Train or Test set: Average loss: 0.8436, Accuracy: 5796/8000 (72.45%)
Size Prev (MB): 0.567652, Size Current (MB): 0.212575, Comp Rate: 2.670361049041515 
QConfig(activation=functools.partial(<class 'torch.quantization.observer.HistogramObserver'>, reduce_range=True), weight=functools.partial(<class 'torch.quantization.observer.PerChannelMinMaxObserver'>, dtype=torch.qint8, qscheme=torch.per_channel_symmetric))

Train or Test set: Average loss: 0.3152, Accuracy: 3790/4224 (89.73%)
Post Training Quantization: Calibration done
Post Training Quantization: Convert done

Train or 

In [28]:
getResult4QuantChannel(args,urbansound,train_loader_urbansound,test_loader_urbansound,32,10000)


QConfig(activation=functools.partial(<class 'torch.quantization.observer.HistogramObserver'>, reduce_range=True), weight=functools.partial(<class 'torch.quantization.observer.PerChannelMinMaxObserver'>, dtype=torch.qint8, qscheme=torch.per_channel_symmetric))

Train or Test set: Average loss: 0.0007, Accuracy: 4224/4224 (100.00%)
Post Training Quantization: Calibration done
Post Training Quantization: Convert done

Train or Test set: Average loss: 0.3667, Accuracy: 792/873 (90.72%)
Size Prev (MB): 0.555242, Size Current (MB): 0.209504, Comp Rate: 2.650269207270506 
QConfig(activation=functools.partial(<class 'torch.quantization.observer.HistogramObserver'>, reduce_range=True), weight=functools.partial(<class 'torch.quantization.observer.PerChannelMinMaxObserver'>, dtype=torch.qint8, qscheme=torch.per_channel_symmetric))

Train or Test set: Average loss: 0.0006, Accuracy: 4224/4224 (100.00%)
Post Training Quantization: Calibration done
Post Training Quantization: Convert done

Train or 

In [29]:
getResult4QuantChannel(args,pamap2,train_loader_pamap2,test_loader_pamap2,32,10000)


QConfig(activation=functools.partial(<class 'torch.quantization.observer.HistogramObserver'>, reduce_range=True), weight=functools.partial(<class 'torch.quantization.observer.PerChannelMinMaxObserver'>, dtype=torch.qint8, qscheme=torch.per_channel_symmetric))

Train or Test set: Average loss: 0.0004, Accuracy: 4224/4224 (100.00%)
Post Training Quantization: Calibration done
Post Training Quantization: Convert done

Train or Test set: Average loss: 0.4301, Accuracy: 4678/5209 (89.81%)
Size Prev (MB): 0.826388, Size Current (MB): 0.299445, Comp Rate: 2.7597321711833556 
QConfig(activation=functools.partial(<class 'torch.quantization.observer.HistogramObserver'>, reduce_range=True), weight=functools.partial(<class 'torch.quantization.observer.PerChannelMinMaxObserver'>, dtype=torch.qint8, qscheme=torch.per_channel_symmetric))

Train or Test set: Average loss: 0.0005, Accuracy: 4224/4224 (100.00%)
Post Training Quantization: Calibration done
Post Training Quantization: Convert done

Train 

In [30]:
getResult4QuantChannel(args,skoda,train_loader_skoda,test_loader_skoda,32,10000)


QConfig(activation=functools.partial(<class 'torch.quantization.observer.HistogramObserver'>, reduce_range=True), weight=functools.partial(<class 'torch.quantization.observer.PerChannelMinMaxObserver'>, dtype=torch.qint8, qscheme=torch.per_channel_symmetric))

Train or Test set: Average loss: 0.0002, Accuracy: 4224/4224 (100.00%)
Post Training Quantization: Calibration done
Post Training Quantization: Convert done

Train or Test set: Average loss: 0.2935, Accuracy: 1099/1193 (92.12%)
Size Prev (MB): 0.77635, Size Current (MB): 0.283962, Comp Rate: 2.7339925764714996 
QConfig(activation=functools.partial(<class 'torch.quantization.observer.HistogramObserver'>, reduce_range=True), weight=functools.partial(<class 'torch.quantization.observer.PerChannelMinMaxObserver'>, dtype=torch.qint8, qscheme=torch.per_channel_symmetric))

Train or Test set: Average loss: 0.0002, Accuracy: 4224/4224 (100.00%)
Post Training Quantization: Calibration done
Post Training Quantization: Convert done

Train o

## QAT Results

In [7]:
getResult4qaQuantChannel(args,hhar,train_loader_hhar,test_loader_hhar,32,1000)


QConfig(activation=functools.partial(<class 'torch.quantization.fake_quantize.FakeQuantize'>, observer=<class 'torch.quantization.observer.MovingAverageMinMaxObserver'>, quant_min=0, quant_max=255, reduce_range=True), weight=functools.partial(<class 'torch.quantization.fake_quantize.FakeQuantize'>, observer=<class 'torch.quantization.observer.MovingAveragePerChannelMinMaxObserver'>, quant_min=-128, quant_max=127, dtype=torch.qint8, qscheme=torch.per_channel_symmetric, reduce_range=False, ch_axis=0))


  0%|          | 0/465 [00:00<?, ?it/s]


Test set: Average loss: 6.9558, Accuracy: 2016/4224 (47.73%)
Post Training Quantization: Calibration done
Epoch:     1


100%|██████████| 465/465 [02:02<00:00,  3.80it/s]



Test set: Average loss: 1.5208, Accuracy: 5236/7721 (67.82%)
New Best Accuracy: (67.82%) at Epoch: (1)



...


Test set: Average loss: 3.4340, Accuracy: 3416/7721 (44.24%)
New Best Accuracy: (44.24%) at Epoch: (1)



  0%|          | 0/465 [00:00<?, ?it/s]


Test set: Average loss: 3.7948, Accuracy: 3186/7721 (41.26%)
MY Current Accuracy: 41.26408496308768, at Epoch: 6
Epoch:     1


100%|██████████| 465/465 [00:52<00:00,  8.92it/s]



Test set: Average loss: 1.3505, Accuracy: 5726/7721 (74.16%)
New Best Accuracy: (74.16%) at Epoch: (1)


Test set: Average loss: 1.4017, Accuracy: 5566/7721 (72.09%)
MY Current Accuracy: 72.08910762854552, at Epoch: 7
Post Training Quantization: Convert done

Test set: Average loss: 1.4017, Accuracy: 5566/7721 (72.09%)
Size Prev (MB): 0.844102, Size Current (MB): 0.305275, Comp Rate: 2.765054459094259 
[[47.72727273 70.78098692  2.76504964]
 [55.94223485 48.12848077  2.76506602]
 [58.47537879 31.36899365  2.76508414]
 [56.91287879 41.09571299  2.76510225]
 [48.53219697 72.08910763  2.76505446]]
Accuracy: 52.6926563916591, Acc_Std: 16.207587172623345, Comp Rate Avg.: 2.7650713036975914, Comp Rate Std.: 1.951112963898013e-05


In [11]:
getResult4qaQuantChannel(args,hhar,train_loader_hhar,test_loader_hhar,32,1000)


QConfig(activation=functools.partial(<class 'torch.quantization.fake_quantize.FakeQuantize'>, observer=<class 'torch.quantization.observer.MovingAverageMinMaxObserver'>, quant_min=0, quant_max=255, reduce_range=True), weight=functools.partial(<class 'torch.quantization.fake_quantize.FakeQuantize'>, observer=<class 'torch.quantization.observer.MovingAveragePerChannelMinMaxObserver'>, quant_min=-128, quant_max=127, dtype=torch.qint8, qscheme=torch.per_channel_symmetric, reduce_range=False, ch_axis=0))


  0%|          | 0/465 [00:00<?, ?it/s]


Test set: Average loss: 6.9558, Accuracy: 2016/4224 (47.73%)
Post Training Quantization: Calibration done
Epoch:     1


  0%|          | 0/465 [00:00<?, ?it/s]


Test set: Average loss: 1.5208, Accuracy: 5236/7721 (67.82%)
New Best Accuracy: (67.82%) at Epoch: (1)

Epoch:     2


...


Test set: Average loss: 2.1190, Accuracy: 4978/7721 (64.47%)
New Best Accuracy: (64.47%) at Epoch: (1)

Epoch:     2


  0%|          | 0/465 [00:00<?, ?it/s]


Test set: Average loss: 2.1783, Accuracy: 5053/7721 (65.44%)
New Best Accuracy: (65.44%) at Epoch: (2)

Epoch:     3


100%|██████████| 465/465 [01:20<00:00,  5.78it/s]



Test set: Average loss: 1.5583, Accuracy: 5617/7721 (72.75%)
New Best Accuracy: (72.75%) at Epoch: (3)


Test set: Average loss: 2.0125, Accuracy: 4396/7721 (56.94%)
MY Current Accuracy: 56.935630099728016, at Epoch: 3
Post Training Quantization: Convert done

Test set: Average loss: 2.0125, Accuracy: 4396/7721 (56.94%)
Size Prev (MB): 0.844102, Size Current (MB): 0.305267, Comp Rate: 2.7651269216783994 
[[47.72727273 42.13184821  2.76507257]
 [55.94223485 51.09441782  2.76509069]
 [58.47537879 56.9356301   2.76512692]]
Accuracy: 50.053965375814876, Acc_Std: 6.088234311998486, Comp Rate Avg.: 2.7650967286580754, Comp Rate Std.: 2.2594349440853163e-05


In [14]:
getResult4qaQuantChannel(args,hhar,train_loader_hhar,test_loader_hhar,32,1000)


QConfig(activation=functools.partial(<class 'torch.quantization.fake_quantize.FakeQuantize'>, observer=<class 'torch.quantization.observer.MovingAverageMinMaxObserver'>, quant_min=0, quant_max=255, reduce_range=True), weight=functools.partial(<class 'torch.quantization.fake_quantize.FakeQuantize'>, observer=<class 'torch.quantization.observer.MovingAveragePerChannelMinMaxObserver'>, quant_min=-128, quant_max=127, dtype=torch.qint8, qscheme=torch.per_channel_symmetric, reduce_range=False, ch_axis=0))


  0%|          | 0/465 [00:00<?, ?it/s]


Train or Test set: Average loss: 6.9558, Accuracy: 2016/4224 (47.73%)
Post Training Quantization: Calibration done
Epoch:     1


100%|██████████| 465/465 [01:20<00:00,  5.81it/s]



Train or Test set: Average loss: 0.2083, Accuracy: 1295/1408 (91.97%)

Test set: Average loss: 1.4236, Accuracy: 5497/7721 (71.20%)
New Best Accuracy: (71.20%) at Epoch: (1)



...


Train or Test set: Average loss: 0.1298, Accuracy: 1357/1408 (96.38%)

Test set: Average loss: 1.5606, Accuracy: 5524/7721 (71.55%)
New Best Accuracy: (71.55%) at Epoch: (1)



  0%|          | 0/465 [00:00<?, ?it/s]


Test set: Average loss: 1.6490, Accuracy: 5464/7721 (70.77%)
MY Current Accuracy: 70.76803522859733, at Epoch: 2
Epoch:     1


100%|██████████| 465/465 [01:14<00:00,  6.22it/s]



Train or Test set: Average loss: 0.1061, Accuracy: 1362/1408 (96.73%)

Test set: Average loss: 1.7238, Accuracy: 4747/7721 (61.48%)
New Best Accuracy: (61.48%) at Epoch: (1)


Test set: Average loss: 3.7843, Accuracy: 3394/7721 (43.96%)
MY Current Accuracy: 43.95803652376635, at Epoch: 3
Post Training Quantization: Convert done

Test set: Average loss: 3.7843, Accuracy: 3394/7721 (43.96%)
Size Prev (MB): 0.844096, Size Current (MB): 0.305265, Comp Rate: 2.7651253828640687 
[[47.72727273 74.79601088  2.76509069]
 [55.94223485 48.03781894  2.76506602]
 [58.47537879 43.95803652  2.76512538]]
Accuracy: 55.59728877951906, Acc_Std: 13.677337797330336, Comp Rate Avg.: 2.7650940318823287, Comp Rate Std.: 2.434856273354587e-05


In [21]:
getResult4qaQuantChannel(args,hhar,train_loader_hhar,test_loader_hhar,32,1000)


QConfig(activation=functools.partial(<class 'torch.quantization.fake_quantize.FakeQuantize'>, observer=<class 'torch.quantization.observer.MovingAverageMinMaxObserver'>, quant_min=0, quant_max=255, reduce_range=True), weight=functools.partial(<class 'torch.quantization.fake_quantize.FakeQuantize'>, observer=<class 'torch.quantization.observer.MovingAveragePerChannelMinMaxObserver'>, quant_min=-128, quant_max=127, dtype=torch.qint8, qscheme=torch.per_channel_symmetric, reduce_range=False, ch_axis=0))


  0%|          | 0/465 [00:00<?, ?it/s]


Train or Test set: Average loss: 6.9558, Accuracy: 2016/4224 (47.73%)
Post Training Quantization: Calibration done
Epoch:     1


100%|██████████| 465/465 [01:09<00:00,  6.65it/s]



Train or Test set: Average loss: 0.2083, Accuracy: 1295/1408 (91.97%)

Train or Test set: Average loss: 1.4236, Accuracy: 5497/7721 (71.20%)
New Best Accuracy: (71.20%) at Epoch: (1)



...


Train or Test set: Average loss: 1.1150, Accuracy: 987/1408 (70.10%)

Train or Test set: Average loss: 2.9515, Accuracy: 3628/7721 (46.99%)
New Best Accuracy: (46.99%) at Epoch: (1)



  0%|          | 0/465 [00:00<?, ?it/s]


Train or Test set: Average loss: 4.1408, Accuracy: 3342/7721 (43.28%)
MY Current Accuracy: 43.284548633596685, at Epoch: 2
Epoch:     1


 36%|███▋      | 169/465 [00:24<00:49,  5.93it/s]

KeyboardInterrupt: 

In [6]:
getResult4qaQuantChannel(args,hhar,train_loader_hhar,test_loader_hhar,32,1000)


QConfig(activation=functools.partial(<class 'torch.quantization.fake_quantize.FakeQuantize'>, observer=<class 'torch.quantization.observer.MovingAverageMinMaxObserver'>, quant_min=0, quant_max=255, reduce_range=True), weight=functools.partial(<class 'torch.quantization.fake_quantize.FakeQuantize'>, observer=<class 'torch.quantization.observer.MovingAveragePerChannelMinMaxObserver'>, quant_min=-128, quant_max=127, dtype=torch.qint8, qscheme=torch.per_channel_symmetric, reduce_range=False, ch_axis=0))


  0%|          | 0/465 [00:00<?, ?it/s]


Train or Test set: Average loss: 6.9558, Accuracy: 2016/4224 (47.73%)
Post Training Quantization: Calibration done
Epoch:     1


100%|██████████| 465/465 [00:55<00:00,  8.41it/s]



Train or Test set: Average loss: 0.0500, Accuracy: 1379/1408 (97.94%)

Train or Test set: Average loss: 1.7899, Accuracy: 4919/7721 (63.71%)
New Best Accuracy: (63.71%) at Epoch: (1)



...


Train or Test set: Average loss: 0.0313, Accuracy: 1395/1408 (99.08%)

Train or Test set: Average loss: 1.2592, Accuracy: 5885/7721 (76.22%)
New Best Accuracy: (76.22%) at Epoch: (1)



  0%|          | 0/465 [00:00<?, ?it/s]


Train or Test set: Average loss: 1.3169, Accuracy: 5827/7721 (75.47%)
MY Current Accuracy: 75.46949876958944, at Epoch: 2
Epoch:     1


100%|██████████| 465/465 [01:26<00:00,  5.39it/s]



Train or Test set: Average loss: 0.0292, Accuracy: 1394/1408 (99.01%)

Train or Test set: Average loss: 1.3403, Accuracy: 5472/7721 (70.87%)
New Best Accuracy: (70.87%) at Epoch: (1)


Train or Test set: Average loss: 1.3377, Accuracy: 5573/7721 (72.18%)
New Best Accuracy: (72.18%) at Epoch: (1)

MY Current Accuracy: 72.17976945991452, at Epoch: 3
Post Training Quantization: Convert done

Train or Test set: Average loss: 1.3377, Accuracy: 5573/7721 (72.18%)
Size Prev (MB): 0.844015, Size Current (MB): 0.305164, Comp Rate: 2.7657751241955144 
[[47.72727273 51.81971247  2.76579966]
 [55.94223485 79.80831499  2.7656762 ]
 [58.47537879 72.17976946  2.76577512]]
Accuracy: 67.93593230583258, Acc_Std: 11.813779609570197, Comp Rate Avg.: 2.765750327562096, Comp Rate Std.: 5.33645265358415e-05


In [10]:
getResult4qaQuantChannel(args,hhar,train_loader_hhar,test_loader_hhar,32,1000)


QConfig(activation=functools.partial(<class 'torch.quantization.fake_quantize.FakeQuantize'>, observer=<class 'torch.quantization.observer.MovingAverageMinMaxObserver'>, quant_min=0, quant_max=255, reduce_range=True), weight=functools.partial(<class 'torch.quantization.fake_quantize.FakeQuantize'>, observer=<class 'torch.quantization.observer.MovingAveragePerChannelMinMaxObserver'>, quant_min=-128, quant_max=127, dtype=torch.qint8, qscheme=torch.per_channel_symmetric, reduce_range=False, ch_axis=0))


  0%|          | 0/465 [00:00<?, ?it/s]


Train or Test set: Average loss: 6.9558, Accuracy: 2016/4224 (47.73%)
Post Training Quantization: Calibration done
Epoch:     1


100%|██████████| 465/465 [01:35<00:00,  4.86it/s]



Train or Test set: Average loss: 0.0417, Accuracy: 501/512 (97.85%)

Train or Test set: Average loss: 1.5830, Accuracy: 2436/4000 (60.90%)


...


Train or Test set: Average loss: 0.0352, Accuracy: 506/512 (98.83%)

Train or Test set: Average loss: 3.4808, Accuracy: 2541/4000 (63.52%)


  0%|          | 0/465 [00:00<?, ?it/s]


Train or Test set: Average loss: 2.9405, Accuracy: 5071/7721 (65.68%)
New Best Accuracy: (65.68%) at Epoch: (1)

MY Current Accuracy: 65.67802098173811, at Epoch: 1
Epoch:     1


100%|██████████| 465/465 [01:54<00:00,  4.05it/s]



Train or Test set: Average loss: 0.0357, Accuracy: 506/512 (98.83%)

Train or Test set: Average loss: 1.5784, Accuracy: 2487/4000 (62.17%)

Train or Test set: Average loss: 1.6792, Accuracy: 5114/7721 (66.23%)
New Best Accuracy: (66.23%) at Epoch: (1)

MY Current Accuracy: 66.23494366014765, at Epoch: 2
Post Training Quantization: Convert done

Train or Test set: Average loss: 1.6792, Accuracy: 5114/7721 (66.23%)
Size Prev (MB): 0.844015, Size Current (MB): 0.305173, Comp Rate: 2.7656935574248047 
[[47.72727273 66.54578422  2.76571998]
 [55.94223485 76.46677891  2.76568854]
 [58.47537879 67.02499676  2.76565152]
 [56.91287879 69.57647973  2.76569934]
 [48.53219697 66.23494366  2.76569356]]
Accuracy: 69.16979665846392, Acc_Std: 3.8343268284617533, Comp Rate Avg.: 2.7656905879252554, Comp Rate Std.: 2.226947379776514e-05


In [12]:
getResult4qaQuantChannel(args,ninadb2,train_loader_ninadb2,test_loader_ninadb2,32,1000)


QConfig(activation=functools.partial(<class 'torch.quantization.fake_quantize.FakeQuantize'>, observer=<class 'torch.quantization.observer.MovingAverageMinMaxObserver'>, quant_min=0, quant_max=255, reduce_range=True), weight=functools.partial(<class 'torch.quantization.fake_quantize.FakeQuantize'>, observer=<class 'torch.quantization.observer.MovingAveragePerChannelMinMaxObserver'>, quant_min=-128, quant_max=127, dtype=torch.qint8, qscheme=torch.per_channel_symmetric, reduce_range=False, ch_axis=0))


  0%|          | 0/244 [00:00<?, ?it/s]


Train or Test set: Average loss: 0.7183, Accuracy: 3135/4224 (74.22%)
Post Training Quantization: Calibration done
Epoch:     1


100%|██████████| 244/244 [00:14<00:00, 16.68it/s]



Train or Test set: Average loss: 0.5002, Accuracy: 415/512 (81.05%)

Train or Test set: Average loss: 0.8617, Accuracy: 2755/4000 (68.88%)


...


Train or Test set: Average loss: 0.5884, Accuracy: 376/512 (73.44%)

Train or Test set: Average loss: 0.8649, Accuracy: 2650/4000 (66.25%)


  0%|          | 0/244 [00:00<?, ?it/s]


Train or Test set: Average loss: 0.9324, Accuracy: 4325/6381 (67.78%)
New Best Accuracy: (67.78%) at Epoch: (1)

MY Current Accuracy: 67.77934493026171, at Epoch: 1
Epoch:     1


100%|██████████| 244/244 [00:18<00:00, 13.04it/s]



Train or Test set: Average loss: 0.4700, Accuracy: 417/512 (81.45%)

Train or Test set: Average loss: 0.8249, Accuracy: 2751/4000 (68.78%)

Train or Test set: Average loss: 0.8633, Accuracy: 4282/6381 (67.11%)
New Best Accuracy: (67.11%) at Epoch: (1)

MY Current Accuracy: 67.10546936216893, at Epoch: 2
Post Training Quantization: Convert done

Train or Test set: Average loss: 0.8633, Accuracy: 4282/6381 (67.11%)
Size Prev (MB): 0.115397, Size Current (MB): 0.045661, Comp Rate: 2.527255206850485 
[[74.21875    65.71070365  2.52680331]
 [78.17234848 70.5375333   2.52669148]
 [58.61742424 63.61071932  2.52718831]
 [70.43087121 62.90550071  2.52722176]
 [77.46212121 67.10546936  2.52725521]]
Accuracy: 65.97398526876665, Acc_Std: 2.7267624598549105, Comp Rate Avg.: 2.527032014899871, Comp Rate Std.: 0.00023601492093965604
